In [184]:
from enum import Enum
from random import shuffle, randint
import random

class Card(Enum):
    PIKA_EX = 1
    BASIC = 2
    P_BALL = 3
    P_RE = 4
    X_SP = 5
    OTHER = 6

class BoardState:
    active_slot = []
    bench = []
    energy = False
    basic_energy_count = 0
    pika_energy_count = 0
    damage_counter = 0

class Simulation: 
    
    #Game start: initiate hand and deck shuffled
    def __init__(self, turn):
        self.turn = turn
        self.hand = []
        self.deck = [Card.PIKA_EX] * 2 + \
                    [Card.P_BALL] * 2 + \
                    [Card.P_RE] * 2 + \
                    [Card.X_SP] * 2 + \
                    [Card.BASIC] * 4 + \
                    [Card.OTHER] * 8
        shuffle(self.deck)

    def display_bs(self):
        print(BoardState.active_slot)
        print(BoardState.bench)

    #Remove a card from deck and place it in your hand
    def draw_card(self, times=1):
        for card in range(times):
            self.hand.append(self.deck.pop())

    def turn_0(self):
        
        #Put PIKA_EX or BASIC into the active slot if empty
        if not BoardState.active_slot:
            if Card.PIKA_EX in self.hand:
                BoardState.active_slot = Card.PIKA_EX
                self.hand.remove(Card.PIKA_EX)
            else:
                BoardState.active_slot = Card.BASIC
                self.hand.remove(Card.BASIC)

        #Fill the bench with PIKA_EX/BASIC up to 3
        while len(BoardState.bench) < 3:
            if Card.PIKA_EX in self.hand:
                BoardState.bench.append(Card.PIKA_EX)
                self.hand.remove(Card.PIKA_EX)
            elif Card.BASIC in self.hand and (BoardState.bench + [BoardState.active_slot]).count(Card.BASIC) < 3:   #Won't fill the board with all basics (leaves room for PIKA_EX)
                BoardState.bench.append(Card.BASIC)
                self.hand.remove(Card.BASIC)
            else:
                break

        self.display_bs()

    def player_turn(self):
        
        self.draw_card(1)

        #Energy generation; 1 energy available per turn, with the exception of player turn 1
        if self.turn != 0:
            BoardState.energy = True

        #Play available card draw: first P_RE then P_BALL to maximize chances of pulling PIKA_EX/filling the board
        if Card.P_RE in self.hand:
            print('Playing Professors Research')
            self.hand.remove(Card.P_RE)
            self.draw_card(2)
        while Card.P_BALL in self.hand:
            print('Playing Pokeball')
            self.hand.remove(Card.P_BALL)
            index_choice = random.choice([i for i, x in enumerate(self.deck) if x == Card.BASIC or x == Card.PIKA_EX])
            self.hand.append(self.deck[index_choice])

        #Play newly drawn PIKA_EX or BASIC cards
        while len(BoardState.bench) < 3:
            if Card.PIKA_EX in self.hand:
                BoardState.bench.append(Card.PIKA_EX)
                self.hand.remove(Card.PIKA_EX)
            elif Card.BASIC in self.hand and (BoardState.bench + [BoardState.active_slot]).count(Card.BASIC) < 3:   #Won't fill the board with all basics (leaves room for PIKA_EX)
                BoardState.bench.append(Card.BASIC)
                self.hand.remove(Card.BASIC)
            else:
                break

        self.display_bs()

        #Engage PIKA_EX into active slot if not already, by means of X_SP or existing energy count on a BASIC
        if BoardState.active_slot != Card.PIKA_EX and Card.PIKA_EX in BoardState.bench:
            if Card.X_SP in self.hand:
                print('Playing X-Speed')
                self.hand.remove(Card.X_SP)
                BoardState.active_slot = Card.PIKA_EX
                BoardState.bench.remove(Card.PIKA_EX)
                BoardState.bench.append(Card.BASIC)
            elif BoardState.basic_energy_count > 0:
                BoardState.basic_energy_count = 0
                BoardState.active_slot = Card.PIKA_EX
                BoardState.bench.remove(Card.PIKA_EX)
                BoardState.bench.append(Card.BASIC)
        
        #Place energy on whatever is in the active slot, UNLESS it is a BASIC and already has one
        while BoardState.energy == True:
            if Card.PIKA_EX == BoardState.active_slot:
                BoardState.pika_energy_count += 1
            elif Card.BASIC == BoardState.active_slot:
                BoardState.basic_energy_count += 1
            BoardState.energy = False

        #Attack when PIKA_EX reaches 2 energy
        if BoardState.pika_energy_count >= 2:
            BoardState.damage_counter += len(BoardState.bench) * 30

        self.display_bs()
        print('Pika energy count: ', BoardState.pika_energy_count)
        print('Basic energy count ', BoardState.basic_energy_count)
        print('Damage: ', BoardState.damage_counter)

        #End turn
        if self.turn == 0:
            self.turn += 1
        self.turn += 1

    

    def runSimulation(self):

        #Draw your starting hand, if PIKA_EX or BASIC isn't drawn, redraw your hand
        while not (Card.PIKA_EX in self.hand or Card.BASIC in self.hand):
            self.hand = []
            shuffle(self.deck)
            self.draw_card(5)
        print('Starting hand: {}'.format(self.hand)) #YOS LATER
        print ('Turn {}'.format(self.turn))

        #Conduct first player turn
        self.turn_0()

        #Play through 5 turns
        for turn in range(0,5):
            self.player_turn()
        




        


Simulation(1).runSimulation()

Starting hand: [<Card.OTHER: 6>, <Card.X_SP: 5>, <Card.BASIC: 2>, <Card.BASIC: 2>, <Card.PIKA_EX: 1>]
Turn 1
Card.PIKA_EX
[<Card.BASIC: 2>, <Card.BASIC: 2>]
Card.PIKA_EX
[<Card.BASIC: 2>, <Card.BASIC: 2>]
Card.PIKA_EX
[<Card.BASIC: 2>, <Card.BASIC: 2>]
Pika energy count:  1
Basic energy count  0
Damage:  0
Card.PIKA_EX
[<Card.BASIC: 2>, <Card.BASIC: 2>]
Card.PIKA_EX
[<Card.BASIC: 2>, <Card.BASIC: 2>]
Pika energy count:  2
Basic energy count  0
Damage:  60
Card.PIKA_EX
[<Card.BASIC: 2>, <Card.BASIC: 2>, <Card.BASIC: 2>]
Card.PIKA_EX
[<Card.BASIC: 2>, <Card.BASIC: 2>, <Card.BASIC: 2>]
Pika energy count:  3
Basic energy count  0
Damage:  150
Playing Professors Research
Card.PIKA_EX
[<Card.BASIC: 2>, <Card.BASIC: 2>, <Card.BASIC: 2>]
Card.PIKA_EX
[<Card.BASIC: 2>, <Card.BASIC: 2>, <Card.BASIC: 2>]
Pika energy count:  4
Basic energy count  0
Damage:  240
Playing Pokeball
Card.PIKA_EX
[<Card.BASIC: 2>, <Card.BASIC: 2>, <Card.BASIC: 2>]
Card.PIKA_EX
[<Card.BASIC: 2>, <Card.BASIC: 2>, <Card.BA